In [1]:
from distilabel.dataset import Dataset

ds = Dataset.load_from_disk(
    "data/fml-generation/2024-02-13-10-37-43/checkpoints/responses_wrangled"
)

/home/ben/code/distilabel-workbench/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = ds.to_pandas()

In [3]:
df

,domain,generation_model,generation_prompt,raw_generation_responses,function,parses,models,instructions,generations,__index_level_0__
0,Functions for search APIs,"[gpt-4-1106-preview, gpt-4-1106-preview]",[[{'content': 'You are an AI assistant that th...,"[{\n ""function_calls"": [\n {\n ""name""...","{""function"":{""name"":""performSearch"",""descripti...",True,True,[Retrieve the latest articles regarding the ec...,"[[{""name"":""performSearch"",""arguments"":{""query""...",0
1,Functions for search APIs,"[gpt-4-1106-preview, gpt-4-1106-preview]",[[{'content': 'You are an AI assistant that th...,"[{\n ""function_calls"": [\n {\n ""name""...","{""function"":{""name"":""performSearch"",""descripti...",True,True,[How would one go about finding the nutritiona...,"[[{""name"":""performSearch"",""arguments"":{""query""...",1
2,Functions for search APIs,"[gpt-4-1106-preview, gpt-4-1106-preview]",[[{'content': 'You are an AI assistant that th...,"[\n{\n ""function_calls"": [\n {\n ""nam...","{""function"":{""name"":""performSearch"",""descripti...",True,True,[Retrieve the latest articles related to artif...,"[[{""name"":""performSearch"",""arguments"":{""query""...",2
3,Functions for search APIs,"[gpt-4-1106-preview, gpt-4-1106-preview]",[[{'content': 'You are an AI assistant that th...,"[{\n ""function_calls"": [\n {\n ""name""...","{""function"":{""name"":""performSearch"",""descripti...",True,True,[Could you find these nutrition articles in th...,"[[{""name"":""performSearch"",""arguments"":{""query""...",3
4,Functions for search APIs,"[gpt-4-1106-preview, gpt-4-1106-preview]",[[{'content': 'You are an AI assistant that th...,"[{\n ""function_calls"": [\n {\n ""name""...","{""function"":{""name"":""performSearch"",""descripti...",True,True,[Discover the most relevant articles about ren...,"[[{""name"":""performSearch"",""arguments"":{""query""...",4
...,...,...,...,...,...,...,...,...,...,...
2019,Functions for task management,"[gpt-4-1106-preview, gpt-4-1106-preview]",[[{'content': 'You are an AI assistant that th...,None,"{'title': 'Task Management System', 'type': 'o...",None,None,[Show me a summary of completed tasks versus p...,"[[{'name': 'get_task_summary', 'arguments': {'...",1719
2020,Functions for task management,"[gpt-4-1106-preview, gpt-4-1106-preview]",[[{'content': 'You are an AI assistant that th...,None,"{'title': 'Task Management System', 'type': 'o...",None,None,[Show me all upcoming deadlines for the market...,"[[{'name': 'get_upcoming_deadlines', 'argument...",1720
2021,Functions for task management,"[gpt-4-1106-preview, gpt-4-1106-preview]",[[{'content': 'You are an AI assistant that th...,None,"{'title': 'Task Management System', 'type': 'o...",None,None,[Which upcoming deadlines should I be aware of...,"[[{'name': 'get_upcoming_deadlines', 'argument...",1721
2022,Functions for task management,"[gpt-4-1106-preview, gpt-4-1106-preview]",[[{'content': 'You are an AI assistant that th...,None,"{'title': 'Task Management System', 'type': 'o...",None,None,[Would it be possible to remind me of deadline...,"[[{'name': 'remind_deadlines', 'arguments': {'...",1722


In [8]:
subset = (
    df.explode("instructions")
    .explode("generations")
    .explode("generations")
    .drop_duplicates(subset=["instructions", "generations", "function"])[
        ["instructions", "generations", "function"]
    ]
)
subset

,instructions,generations,function
0,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...","{""function"":{""name"":""performSearch"",""descripti..."
0,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...","{""function"":{""name"":""performSearch"",""descripti..."
0,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...","{""function"":{""name"":""performSearch"",""descripti..."
0,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...","{""function"":{""name"":""performSearch"",""descripti..."
0,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...","{""function"":{""name"":""performSearch"",""descripti..."
...,...,...,...
2021,Which upcoming deadlines should I be aware of ...,"{'name': 'get_upcoming_deadlines', 'arguments'...","{'title': 'Task Management System', 'type': 'o..."
2022,Would it be possible to remind me of deadlines...,"{'name': 'remind_deadlines', 'arguments': {'da...","{'title': 'Task Management System', 'type': 'o..."
2022,Would it be possible to remind me of deadlines...,"{'name': 'get_upcoming_deadlines', 'arguments'...","{'title': 'Task Management System', 'type': 'o..."
2023,Would you be able to compile the outstanding i...,"{'name': 'compile_issues', 'arguments': {'sour...","{'title': 'Task Management System', 'type': 'o..."


# Parsing


## Function Definition


In [41]:
from src.examples import Tool, Function
import json
from ast import literal_eval


def try_parse(f, model):
    try:
        f = json.loads(f)
    except:
        try:
            f = literal_eval(f)
        except:
            return 0
    try:
        if "function" in f:
            f = f["function"]
        model(**f)
        return 2
    except Exception as e:
        # print(e)
        return 1


subset["parses"] = subset.function.apply(lambda f: try_parse(f, Function))

In [42]:
subset

,instructions,generations,function,parses
0,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...","{""function"":{""name"":""performSearch"",""descripti...",2
0,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...","{""function"":{""name"":""performSearch"",""descripti...",2
0,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...","{""function"":{""name"":""performSearch"",""descripti...",2
0,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...","{""function"":{""name"":""performSearch"",""descripti...",2
0,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...","{""function"":{""name"":""performSearch"",""descripti...",2
...,...,...,...,...
2021,Which upcoming deadlines should I be aware of ...,"{'name': 'get_upcoming_deadlines', 'arguments'...","{'title': 'Task Management System', 'type': 'o...",1
2022,Would it be possible to remind me of deadlines...,"{'name': 'remind_deadlines', 'arguments': {'da...","{'title': 'Task Management System', 'type': 'o...",1
2022,Would it be possible to remind me of deadlines...,"{'name': 'get_upcoming_deadlines', 'arguments'...","{'title': 'Task Management System', 'type': 'o...",1
2023,Would you be able to compile the outstanding i...,"{'name': 'compile_issues', 'arguments': {'sour...","{'title': 'Task Management System', 'type': 'o...",1


In [43]:
subset[subset.parses != 2].parses.value_counts()

1    1607
Name: parses, dtype: int64

In [44]:
subset[subset.parses != 2]

,instructions,generations,function,parses
20,How chilly is it in Toronto with measurements ...,"{""name"":""getCurrentWeather"",""arguments"":{""loca...","{'function': {'name': 'getCurrentWeather', 'de...",1
20,How chilly is it in Toronto with measurements ...,"{""name"":""getCurrentWeather"",""arguments"":{""loca...","{'function': {'name': 'getCurrentWeather', 'de...",1
20,How chilly is it in Toronto with measurements ...,"{""name"":""getCurrentWeather"",""arguments"":{""loca...","{'function': {'name': 'getCurrentWeather', 'de...",1
20,How chilly is it in Toronto with measurements ...,"{""name"":""getCurrentWeather"",""arguments"":{""loca...","{'function': {'name': 'getCurrentWeather', 'de...",1
20,How chilly is it in Toronto with measurements ...,"{""name"":""getCurrentWeather"",""arguments"":{""loca...","{'function': {'name': 'getCurrentWeather', 'de...",1
...,...,...,...,...
2021,Which upcoming deadlines should I be aware of ...,"{'name': 'get_upcoming_deadlines', 'arguments'...","{'title': 'Task Management System', 'type': 'o...",1
2022,Would it be possible to remind me of deadlines...,"{'name': 'remind_deadlines', 'arguments': {'da...","{'title': 'Task Management System', 'type': 'o...",1
2022,Would it be possible to remind me of deadlines...,"{'name': 'get_upcoming_deadlines', 'arguments'...","{'title': 'Task Management System', 'type': 'o...",1
2023,Would you be able to compile the outstanding i...,"{'name': 'compile_issues', 'arguments': {'sour...","{'title': 'Task Management System', 'type': 'o...",1


## Function Response


In [45]:
from src.examples import FunctionCallResponse

subset["generations_parses"] = subset.generations.apply(
    lambda f: try_parse(f, FunctionCallResponse)
)

In [46]:
subset

,instructions,generations,function,parses,generations_parses
0,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...","{""function"":{""name"":""performSearch"",""descripti...",2,2
0,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...","{""function"":{""name"":""performSearch"",""descripti...",2,2
0,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...","{""function"":{""name"":""performSearch"",""descripti...",2,2
0,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...","{""function"":{""name"":""performSearch"",""descripti...",2,2
0,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...","{""function"":{""name"":""performSearch"",""descripti...",2,2
...,...,...,...,...,...
2021,Which upcoming deadlines should I be aware of ...,"{'name': 'get_upcoming_deadlines', 'arguments'...","{'title': 'Task Management System', 'type': 'o...",1,2
2022,Would it be possible to remind me of deadlines...,"{'name': 'remind_deadlines', 'arguments': {'da...","{'title': 'Task Management System', 'type': 'o...",1,2
2022,Would it be possible to remind me of deadlines...,"{'name': 'get_upcoming_deadlines', 'arguments'...","{'title': 'Task Management System', 'type': 'o...",1,2
2023,Would you be able to compile the outstanding i...,"{'name': 'compile_issues', 'arguments': {'sour...","{'title': 'Task Management System', 'type': 'o...",1,2


In [47]:
subset[subset.parses != 2].parses.value_counts()

1    1607
Name: parses, dtype: int64